# Text Classification

- Ensemble 10 mini-model -> voting

### Library

In [ ]:
from tqdm import tqdm
import transformers
import pandas as pd
import numpy as np
import os

from transformers import AutoTokenizer
from datasets import load_dataset, load_metric
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import pipeline

### GPU settings

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "0,1,2,3,4,5,6,7"

### Hyperparameters

In [ ]:
SAMPLE_RATIO = 0.01
TEST_RATIO = 0.1
BACKBONE_MODEL = "klue/roberta-large"
BATCH_SIZE = 4
LEARNING_RATE = 3e-5
MAX_EPOCHS = 1
WEIGHT_DECAY = 0.01

### Dataset

In [ ]:
df = pd.read_csv("train.csv")

In [ ]:
df = df.dropna()

In [ ]:
df = df.fillna("")

In [ ]:
df = df.sample(int(len(df)*SAMPLE_RATIO))

In [ ]:
df = df.rename(columns={"data": "sentence", "category": "label"})

In [ ]:
label_list = sorted(list(set(list(df["label"]))))
label_list

In [ ]:
train_df = df[:int(len(df)*(1-TEST_RATIO))]
val_df = df[int(len(df)*(1-TEST_RATIO)):]

In [ ]:
train_df.to_csv("./train_df.csv", index=False)
val_df.to_csv("./val_df.csv", index=False)

In [ ]:
train_df

In [ ]:
val_df

In [ ]:
dataset = load_dataset('csv', data_files={'train': './train_df.csv','test': './val_df.csv'})

In [ ]:
dataset

In [ ]:
dataset["train"][0]

In [ ]:
dataset["test"][0]

### Metrics

In [ ]:
metric = load_metric("f1")

In [ ]:
metric

In [ ]:
import numpy as np
fake_preds = np.random.randint(0, 3, size=(64,))
fake_labels = np.random.randint(0, 3, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels, average="macro")

### Preprocess

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(BACKBONE_MODEL, use_fast=True)

In [ ]:
tokenizer("이순신은 조선 중기의 무신이다.", "대한민국은 자유민주주의 국가이다.")

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True)

In [ ]:
preprocess_function(dataset['train'][:5])

In [ ]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

### Training

In [ ]:
num_labels = len(label_list)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(BACKBONE_MODEL, num_labels=num_labels)

In [ ]:
metric_name = "f1"

In [ ]:
model_name = BACKBONE_MODEL.split("/")[-1]

In [ ]:
args = TrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=MAX_EPOCHS,
    weight_decay=WEIGHT_DECAY,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=False,
)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels, average="macro")

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

### Inference

In [ ]:
submission_df = pd.read_csv("./test.csv")[:100]

In [ ]:
sentences = list(submission_df["data"])

In [ ]:
preds = list()

In [ ]:
inferencer = pipeline(task="text-classification", model=model, tokenizer=tokenizer, device=0)
for sentence in tqdm(sentences) : 
    out = inferencer(sentence, batch_size=64, truncation="only_first")
    preds.append(out[0]["label"].split("_")[1])

In [ ]:
submission_df["category"] = preds

In [ ]:
submission_df = submission_df[["index", "category"]]

In [ ]:
submission_df.to_csv("./submission.csv", index=False)